In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import mean_squared_error

In [29]:
X_train = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\X_train_27022019.csv")
y_train = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\y_train_27022019.csv")
X_dev = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\X_dev_27022019.csv")
y_dev = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\y_dev_27022019.csv")
X_test = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\X_test_27022019.csv")
y_test = pd.read_csv(r"C:\Users\malmagor\Desktop\פרויקט גמר לימודים\סופי\y_test_27022019.csv")


In [30]:
X_dev['IsOrganic'] = X_dev['IsOrganic'].astype('int')

In [31]:
X_train['IsOrganic'] = X_train['IsOrganic'].astype('int')

In [32]:
X_test['IsOrganic'] = X_test['IsOrganic'].astype('int')

In [33]:
X_dev['order_HourGroup'] =X_dev['order_HourGroup'].astype('category')


In [34]:
X_train['order_HourGroup'] =X_train['order_HourGroup'].astype('category')

In [35]:
X_test['order_HourGroup'] =X_test['order_HourGroup'].astype('category')

In [36]:
def one_hot_encoding(data,var,drop=True):
    #res = []
    #nm = data[var].name
    nn = data[var].unique()
    for i in nn:
        s = '%s_%s' % (var,i)
        data[s] = 0
        data.loc[data[var]==i,s] = 1
        #res.append(newx)
    if(drop==True):
        data = data.drop(var,axis=1)
    return(data)

In [37]:
X_train = one_hot_encoding(X_train,'order_HourGroup')

X_dev = one_hot_encoding(X_dev,'order_HourGroup')

X_test = one_hot_encoding(X_test,'order_HourGroup')


# The following function calculates the Hamming score that calculates the multilabel accuracy. 

In [38]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)



##  Which models could be ran for multilable outcomes?
 
Following the recommendations of sklearn (https://scikit-learn.org/stable/modules/multiclass.html) the following models accept multilabel outcomes:
 
Support multilabel:
- sklearn.linear_model.RidgeClassifierCV
- sklearn.tree.DecisionTreeClassifier
- sklearn.tree.ExtraTreeClassifier
- sklearn.ensemble.ExtraTreesClassifier
- sklearn.ensemble.RandomForestClassifier
- sklearn.neighbors.KNeighborsClassifier
- sklearn.neighbors.RadiusNeighborsClassifier
 
Also, there is a technique called label powerset (in skmultilearn) that is able to take possible correlations between class labels into account.

In [39]:
#import sys
#!{sys.executable} -m pip install scikit-multilearn

In [40]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.linear_model import LogisticRegression


## THIS IS THE PART THAT YOU MISS... THE MULTILEVEL-Y HAS TO BE TRANSFORMED INTO A NUMPY ARRAY (WITHOUT THE PRODUCT CODE)...

In [41]:
y_dev.head()
ydev = y_dev.copy()
ydev = np.array(ydev)
ydev = np.delete(ydev, 0, 1) 
print(ydev.shape)
print(ydev)

(402512, 30)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [42]:
y_test.head()
ytest = y_test.copy()
ytest = np.array(ytest)
ytest = np.delete(ytest, 0, 1) 
print(ytest.shape)
print(ytest)

(402512, 30)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [43]:
y_train.head()
ytrain = y_train.copy()
ytrain = np.array(ytrain)
ytrain = np.delete(ytrain, 0, 1) 
print(ytrain.shape)
print(ytrain)

(3220100, 30)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# deleting the first column in X_dev/train/test, because we can't put the user_id in the models

In [44]:

Xdev = X_dev.copy()
Xdev = Xdev.drop(['Unnamed: 0'],axis=1)
print(Xdev.shape)
Xdev.head()

(402512, 10)


,reordered,product_popularity_cnt,department_popularity_cnt,IsOrganic,days_previous_Group,Percentage_amount_product,order_HourGroup_18-24,order_HourGroup_06-12,order_HourGroup_00-06,order_HourGroup_12-18
0,1,0.024617,0.929434,1,1,0.0625,1,0,0,0
1,1,0.020455,0.929434,1,1,0.1875,0,1,0,0
2,1,0.115153,0.929434,0,2,0.2500,0,0,1,0
3,1,0.115153,0.929434,0,4,0.0625,0,0,0,1
4,1,0.064592,0.929434,1,4,0.0625,1,0,0,0


In [45]:

Xtrain = X_train.copy()
Xtrain = Xtrain.drop(['Unnamed: 0'],axis=1)
print(Xtrain.shape)
Xtrain.head()

(3220100, 10)


,reordered,product_popularity_cnt,department_popularity_cnt,IsOrganic,days_previous_Group,Percentage_amount_product,order_HourGroup_12-18,order_HourGroup_18-24,order_HourGroup_06-12,order_HourGroup_00-06
0,1,0.092567,0.929434,1,1,0.1250,1,0,0,0
1,0,0.019529,0.929434,0,4,0.3750,1,0,0,0
2,1,0.043180,0.929434,1,2,0.1875,1,0,0,0
3,0,0.017755,0.929434,1,4,0.4375,1,0,0,0
4,1,0.018518,0.051992,1,4,0.2500,1,0,0,0


In [46]:

Xtest = X_test.copy()
Xtest = Xtest.drop(['Unnamed: 0'],axis=1)
print(Xtest.shape)
Xtest.head()

(402512, 10)


,reordered,product_popularity_cnt,department_popularity_cnt,IsOrganic,days_previous_Group,Percentage_amount_product,order_HourGroup_06-12,order_HourGroup_12-18,order_HourGroup_18-24,order_HourGroup_00-06
0,0,0.058997,0.929434,1,3,0.0625,1,0,0,0
1,1,0.017755,0.929434,1,2,0.1875,1,0,0,0
2,0,0.035028,0.929434,0,4,0.1875,0,1,0,0
3,1,0.037687,0.929434,0,1,0.2500,0,1,0,0
4,0,0.020333,0.929434,0,1,0.2500,0,1,0,0


# Logistic Regression 

# ALL THE MODELS MUST BE ENVELOPED INTO THE FUNCTION 
 
# "LabelPowerset()" . 
 
### This function convert the model to a multi-label format. Without it, some models will fail to understand that you are working with a multilabel outcome!

In [19]:
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())


In [20]:
# train
classifier.fit(Xtrain,ytrain)


LabelPowerset(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       require_dense=[True, True])

In [21]:
# predict
predictions = classifier.predict(Xdev)

In [22]:
predictions.shape

(402512, 30)

# The output prediction is in a special format called "lil_matrix" (list of lists format). 
 
### This is a sparce matrix that enables the model to work faster. However, the format is not recognized by many of the model operators. We need to convert them back to an numpy array...

In [23]:
type(predictions)

scipy.sparse.lil.lil_matrix

In [24]:
predictions2 = np.array(predictions.todense())
predictions2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# By summing the columns of the multilabel outcome and prediction we can get a sense of what is going on and how the model is behaving....

In [25]:
print(np.sum(ydev,axis=0))
print(ydev.shape)

[ 7060  8275  8554  8271 37375 14213  7396 25961  7069 23754 11316 46049
 10615 13719 13604 13616  8756  8048  7210  9863  8248  7710  7395 10502
  7906 20866 15349 16773  7616  9423]
(402512, 30)


In [26]:
print(np.sum(predictions2,axis=0))
print(predictions2.shape)

[ 4568  3895   590     7 37375 29588 12931 50548  2808 33550   269 46049
  7483  6736 13604 31538  8488  5318  1213 19222  4687  4338  7395   499
  4129 16714 21940 14325  7616  5089]
(402512, 30)


# And we calculate the accuracy using the score() function and/or the hamming score (see at the begining of the notebook).

In [27]:
classifier.score(Xdev,ydev)

0.4404812775768176

In [28]:
ham=hamming_score(ydev, predictions2)
ham

0.4404812775768176

In [29]:
### We will create a table for comparing the results
#res = pd.DataFrame(columns=['model','accuracy', 'error'])
res = []

In [30]:
res.append({'model':'Logistic Regression ','accuracy':ham})
res

[{'model': 'Logistic Regression ', 'accuracy': 0.4404812775768176}]

# Ridge - Logistic Regression with L2 regularization


# sklearn.linear_model.RidgeClassifierCV

In [40]:
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
# Initialize the predictive model object
#mod_logistic = LogisticRegression()

In [42]:
# Train the model using the training sets
#mod_logistic.fit(X_train, y_train)
mod = LabelPowerset(RidgeClassifierCV())
mod.fit(Xtrain,ytrain)

LabelPowerset(classifier=RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ]), class_weight=None,
         cv=None, fit_intercept=True, normalize=False, scoring=None,
         store_cv_values=False),
       require_dense=[True, True])

In [49]:
# Make predictions using the testing set
pred_Ridge = mod.predict(Xdev)

In [53]:
print(type(pred_Ridge))
print(pred_Ridge.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [54]:
pred_Ridge2 = np.array(pred_Ridge.todense())
print(pred_Ridge2.shape)

(402512, 30)


In [55]:
pred_Ridge2 = np.array(pred_Ridge.todense())
pred_Ridge2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [56]:
#print(np.sum(pred2,axis=1))
print(np.sum(pred_Ridge2,axis=0))
print(pred_Ridge2.shape)

[     0      0      0      0 135272  11140      0  21876      0  44724
      0  89352      0      0  28615      0      0      0      0    482
      0      0      0      0      0  45458  21524   4069      0      0]
(402512, 30)


In [57]:
mod.score(Xdev,ydev)

0.24100151051397226

In [58]:
ham=hamming_score(ydev, pred_Ridge2)
ham

0.24100151051397226

In [59]:
res.append({'model':'Ridge - Logistic Regression ','accuracy':ham})
res

[{'model': 'Logistic Regression ', 'accuracy': 0.4404812775768176},
 {'model': 'Ridge - Logistic Regression ', 'accuracy': 0.24100151051397226}]

# sklearn.tree.DecisionTreeClassifier

In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [48]:

mod_DecisionTree = LabelPowerset(DecisionTreeClassifier())


In [49]:
mod_DecisionTree.fit(Xtrain,ytrain)

LabelPowerset(classifier=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       require_dense=[True, True])

In [50]:
# Make predictions 
pred_DecisionTree = mod_DecisionTree.predict(Xdev)

In [65]:
print(type(pred_DecisionTree))
print(pred_DecisionTree.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [66]:
pred_DecisionTree2 = np.array(pred_DecisionTree.todense())
pred_DecisionTree2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [67]:
#print(np.sum(pred2,axis=1))
print(np.sum(pred_DecisionTree2,axis=0))
print(pred_DecisionTree2.shape)

[ 7060  8275  8554  8271 37375 14213  7396 25961  7069 23754 11316 46049
 10615 13719 13604 13616  8756  8048  7210  9863  8248  7710  7395 10502
  7906 20866 15349 16773  7616  9423]
(402512, 30)


In [70]:
ham=hamming_score(ydev, pred_DecisionTree2)
ham

1.0

we have a perfect match then we get one!

In [69]:
res.append({'model':'Decision Tree Classifier ','accuracy':ham})
res

[{'model': 'Logistic Regression ', 'accuracy': 0.4404812775768176},
 {'model': 'Ridge - Logistic Regression ', 'accuracy': 0.24100151051397226},
 {'model': 'Decision Tree Classifier ', 'accuracy': 1.0}]

# sklearn.tree.ExtraTreeClassifier

In [71]:
from sklearn.tree import ExtraTreeClassifier

In [73]:
mod_ExtraTree = LabelPowerset(ExtraTreeClassifier())

In [74]:
mod_ExtraTree.fit(Xtrain,ytrain)

LabelPowerset(classifier=ExtraTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=None,
          splitter='random'),
       require_dense=[True, True])

In [76]:
# Make predictions 
pred_ExtraTree = mod_ExtraTree.predict(Xdev)

In [77]:
print(type(pred_ExtraTree))
print(pred_ExtraTree.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [78]:
pred_ExtraTree2 = np.array(pred_ExtraTree.todense())
pred_ExtraTree2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [79]:
#print(np.sum(pred2,axis=1))
print(np.sum(pred_ExtraTree2,axis=0))
print(pred_ExtraTree2.shape)

[ 7050  8297  8547  8260 37374 14209  7402 25959  7063 23762 11318 46049
 10628 13723 13603 13609  8742  8049  7206  9856  8285  7704  7395 10511
  7909 20860 15349 16770  7615  9408]
(402512, 30)


In [80]:
ham=hamming_score(ydev,pred_ExtraTree2)
ham

0.9995205111897285

In [81]:
res.append({'model':'Decision Tree Classifier ','accuracy':ham})
res

[{'model': 'Logistic Regression ', 'accuracy': 0.4404812775768176},
 {'model': 'Ridge - Logistic Regression ', 'accuracy': 0.24100151051397226},
 {'model': 'Decision Tree Classifier ', 'accuracy': 1.0},
 {'model': 'Decision Tree Classifier ', 'accuracy': 0.9995205111897285}]

# sklearn.ensemble.ExtraTreesClassifier

In [82]:
from sklearn.ensemble import ExtraTreesClassifier

In [83]:
mod_ensemble_ExtraTree = LabelPowerset(ExtraTreeClassifier())

In [84]:
mod_ensemble_ExtraTree.fit(Xtrain,ytrain)

LabelPowerset(classifier=ExtraTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=None,
          splitter='random'),
       require_dense=[True, True])

In [85]:
# predict
pred_ensemble_ExtraTree = mod_ensemble_ExtraTree.predict(Xdev)

In [86]:
print(type(pred_ensemble_ExtraTree))
print(pred_ensemble_ExtraTree.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [87]:
pred_ensemble_ExtraTree2 = np.array(pred_ensemble_ExtraTree.todense())
pred_ensemble_ExtraTree2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [88]:

print(np.sum(pred_ensemble_ExtraTree2,axis=0))
print(pred_ensemble_ExtraTree2.shape)

[ 7061  8275  8551  8270 37374 14213  7392 25963  7059 23758 11320 46049
 10631 13719 13604 13610  8748  8061  7212  9862  8258  7703  7394 10497
  7906 20866 15349 16768  7616  9423]
(402512, 30)


In [89]:
ham=hamming_score(ydev,pred_ensemble_ExtraTree2)
ham

0.9995801367412649

In [90]:
res.append({'model':'ensemble-Decision Tree Classifier ','accuracy':ham})
res

[{'model': 'Logistic Regression ', 'accuracy': 0.4404812775768176},
 {'model': 'Ridge - Logistic Regression ', 'accuracy': 0.24100151051397226},
 {'model': 'Decision Tree Classifier ', 'accuracy': 1.0},
 {'model': 'Decision Tree Classifier ', 'accuracy': 0.9995205111897285},
 {'model': 'ensemble-Decision Tree Classifier ',
  'accuracy': 0.9995801367412649}]

# sklearn.ensemble.RandomForestClassifier

In [91]:
from sklearn.ensemble import RandomForestClassifier


In [92]:
# initialize label powerset multi-label classifier
mod_RandomForest = LabelPowerset(RandomForestClassifier())

In [93]:
mod_RandomForest.fit(Xtrain,ytrain)

F:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


LabelPowerset(classifier=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       require_dense=[True, True])

In [94]:
# predict
pred_RandomForest = mod_RandomForest.predict(Xdev)

In [95]:
print(type(pred_RandomForest))
print(pred_RandomForest.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [96]:
pred_RandomForest2 = np.array(pred_RandomForest.todense())
pred_RandomForest2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [97]:
print(np.sum(pred_ensemble_ExtraTree2,axis=0))
print(pred_ensemble_ExtraTree2.shape)

[ 7061  8275  8551  8270 37374 14213  7392 25963  7059 23758 11320 46049
 10631 13719 13604 13610  8748  8061  7212  9862  8258  7703  7394 10497
  7906 20866 15349 16768  7616  9423]
(402512, 30)


In [98]:
ham=hamming_score(ydev,pred_ensemble_ExtraTree2)
ham

0.9995801367412649

In [99]:
res.append({'model':'Random Forest ','accuracy':ham})
res

[{'model': 'Logistic Regression ', 'accuracy': 0.4404812775768176},
 {'model': 'Ridge - Logistic Regression ', 'accuracy': 0.24100151051397226},
 {'model': 'Decision Tree Classifier ', 'accuracy': 1.0},
 {'model': 'Decision Tree Classifier ', 'accuracy': 0.9995205111897285},
 {'model': 'ensemble-Decision Tree Classifier ',
  'accuracy': 0.9995801367412649},
 {'model': 'Random Forest ', 'accuracy': 0.9995801367412649}]

# sklearn.neighbors.KNeighborsClassifier

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
# initialize label powerset multi-label classifier
mod_KNeighbors = LabelPowerset(KNeighborsClassifier(n_neighbors=3))

In [33]:
mod_KNeighbors.fit(Xtrain,ytrain)

LabelPowerset(classifier=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform'),
       require_dense=[True, True])

In [34]:
# predict
pred_KNeighbors = mod_KNeighbors.predict(Xdev)

In [35]:
print(type(pred_KNeighbors))
print(pred_KNeighbors.shape)

<class 'scipy.sparse.lil.lil_matrix'>
(402512, 30)


In [36]:
pred_KNeighbors2 = np.array(pred_KNeighbors.todense())
pred_KNeighbors2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [37]:
print(np.sum(pred_KNeighbors2,axis=0))
print(pred_KNeighbors2.shape)

[ 6999  8280  8590  8298 37371 14222  7398 25957  7155 23760 11341 46046
 10665 13745 13674 13539  8682  8051  7199  9858  8245  7634  7395 10533
  7963 20837 15361 16790  7546  9378]
(402512, 30)


In [38]:
ham=hamming_score(ydev,pred_KNeighbors2)
ham

0.9965019676432006

In [39]:
res.append({'model':'KNeighbors ','accuracy':ham})
res

[{'model': 'Logistic Regression ', 'accuracy': 0.4404812775768176},
 {'model': 'KNeighbors ', 'accuracy': 0.9965019676432006}]